In [ ]:
# DATA LAKE STORAGE - DFS.CORE

# "abfss://<container-name>@<storage-account-name>.dfs.core.windows.net" (Gen 2 Hierarchical Name)
# "wasbs://<container-name>@<storage-account-name>.dfs.core.windows.net" (Blob)
# "fs.azure.account.key.<storage-account>.dfs.core.windows.net":"<storage-account-access-key>" (Extra_configs)


# BLOBO STORAGE - BLOB.CORE

# "abfss://<container-name>@<storage-account-name>.blob.core.windows.net" (Gen 2 Hierarchical Name)
# "wasbs://<container-name>@<storage-account-name>.blob.core.windows.net" (Blob)
# "fs.azure.account.key.<storage-account>.blob.core.windows.net":"<storage-account-access-key>" (Extra_configs)


dbutils.fs.mount(
 source = "wasbs://projetodados@storageprojetoengdados.blob.core.windows.net",
 mount_point = "/mnt/projetodados", 
 extra_configs = {"fs.azure.account.key.storageprojetoengdados.blob.core.windows.net":"L/NL7PxKQ88pOrYwJT1vx+ryH79wo/4TUKN0Ub3yppvzn9DRRH3L4nn7cuPj+fDVeAd4CKrtqX66+AStETjNvg=="}
 )


In [ ]:
# dbutils.fs.unmount("/mnt/projetodados")

In [ ]:
display(dbutils.fs.ls("/mnt/projetodados/raw/dados/2020"))
display(dbutils.fs.ls("/mnt/projetodados/raw/dados/2021"))

In [ ]:
# load = Carregar e criar um dataframe através de um tipo de arquivo, assim passando o caminho do arquivo

df_1_sem_2020 = spark.read.format("csv")\
                          .option("sep",";")\
                          .option("header","True")\
                          .option("inferSchema","True")\
                          .load("/mnt/projetodados/raw/dados/2020/1_sem_2020.csv")

display(df_1_sem_2020)

In [ ]:
df_2_sem_2020 = spark.read.format("csv")\
                          .option("sep",";")\
                          .option("header","True")\
                          .option("inferSchema","True")\
                          .load("/mnt/projetodados/raw/dados/2020/2_sem_2020.csv")

display(df_2_sem_2020)

In [ ]:
df_1_sem_2021 = spark.read.format("csv")\
                          .option("sep",";")\
                          .option("header","True")\
                          .option("inferSchema","True")\
                          .load("/mnt/projetodados/raw/dados/2021/1_sem_2021.csv")

display(df_1_sem_2021)

In [ ]:
df_2_sem_2021 = spark.read.format("csv")\
                          .option("sep",";")\
                          .option("header","True")\
                          .option("inferSchema","True")\
                          .load("/mnt/projetodados/raw/dados/2021/2_sem_2021.csv")
display(df_2_sem_2021)

In [ ]:
# 2 Modos para visualizar:
# display(df_1_sem_2020.dtypes)
# df_1_sem_2020.printSchema()


df_1_sem_2020.printSchema()

In [ ]:
# x = df.union(dataframe_a_se_unido)

df_2020 = df_1_sem_2020.union(df_2_sem_2020)
df_2021 = df_1_sem_2021.union(df_2_sem_2021)
df_full = df_2020.union(df_2021)

In [ ]:
df_processed = df_full.withColumnRenamed("Regiao - Sigla", "REGIAO")\
                   .withColumnRenamed("Estado - Sigla", "ESTADO")\
                   .withColumnRenamed("Municipio", "MUNICIPIO")\
                   .withColumnRenamed("Revenda", "REVENDA")\
                   .withColumnRenamed("CNPJ da Revenda", "CNPJ")\
                   .withColumnRenamed("Nome da Rua", "NOME_RUA")\
                   .withColumnRenamed("Numero Rua", "NUM_RUA")\
                   .withColumnRenamed("Complemento", "COMPL_RUA")\
                   .withColumnRenamed("Bairro", "BAIRRO")\
                   .withColumnRenamed("Cep", "CEP")\
                   .withColumnRenamed("Produto", "PRODUTO")\
                   .withColumnRenamed("Data da Coleta", "DT_COLETA")\
                   .withColumnRenamed("Valor de Venda", "VL_VENDA")\
                   .withColumnRenamed("Valor de Compra", "VL_COMPRA")\
                   .withColumnRenamed("Unidade de Medida", "UNID_MED")\
                   .withColumnRenamed("Bandeira", "BANDEIRA")

display(df_processed)

In [ ]:
df_processed.createOrReplaceTempView("ANP")

In [ ]:
%sql

select * from ANP;

In [ ]:
df_datatypes = spark.sql(""" SELECT REGIAO, 
                                    ESTADO,
                                    MUNICIPIO,
                                    REVENDA,
                                    CNPJ,
                                    NOME_RUA,
                                    NUM_RUA,
                                    COMPL_RUA,
                                    BAIRRO,
                                    CEP,
                                    PRODUTO,
                                    TO_DATE(DT_COLETA, 'dd/mm/yyyy')   as DT_COLETA,
                                    CAST(REPLACE(VL_VENDA, ',' , '.')  as NUMERIC(4,3)) as  VL_VENDA,
                                    CAST(REPLACE(VL_COMPRA,',' , '.')  as NUMERIC(4,3)) AS  VL_COMPRA,
                                    UNID_MED,
                                    BANDEIRA
                                    FROM ANP;
""")

display(df_datatypes)

In [ ]:
df_datatypes.createOrReplaceTempView("ANP_NEW")

In [ ]:
%sql

select * from ANP_NEW;

In [ ]:
# IMPORTANTE: DEVE-SE CRIAR PRIMEIRAMENTE A TABELA NO SQL SERVER PARA FAZER A CARGA DO DATASET NO SQL SERVER


# Verificando quantidade de linhas do daframe

df_datatypes.count()

In [ ]:
# Limitando quantidade de linhas no dataframe com Limit

df_limit = spark.sql("SELECT * FROM ANP_NEW LIMIT 50")

df_limit.count() # 50 Linhas

display(df_limit)

In [ ]:
%sql

-- Vendo o tamanho de uma coluna 
-- len() ou lenght()

-- select max(lenght(BAIRRO)) FROM ANP_NEW;

select max(len(BAIRRO)) FROM ANP_NEW;


In [ ]:
%sql

/*

CREATE TABLE TABLE_ANP_50(
REGIAO VARCHAR(50) NULL,
ESTADO VARCHAR(50) NULL,
MUNICIPIO VARCHAR(50) NULL,
REVENDA VARCHAR(50) NULL,
CNPJ VARCHAR(50) NULL,
NOME_RUA VARCHAR(50) NULL,
NUM_RUA VARCHAR(50) NULL,
COMPL_RUA VARCHAR(50) NULL,
BAIRRO VARCHAR(50) NULL,
CEP VARCHAR(50) NULL,
PRODUTO VARCHAR(50) NULL,
DT_COLETA DATE NULL,
VL_VENDA NUMERIC(4,3) NULL,
VL_COMPRA NUMERIC(4,3) NULL,
UNID_MED VARCHAR(50) NULL,
BANDEIRA VARCHAR(50) NULL
);

*/

In [ ]:
# FAZENDO CARGA DO DATASET NO SQL SERVER 

# dbtable = databricks table (nome da tabela criada no sql server)

# url = url de conexão do servidor sql server --> projeto-eng-dados-sqlserver.database.windows.net

df_limit.write.format("jdbc")\
              .mode("append")\
              .option("url","jdbc:sqlserver://projeto-eng-dados-sqlserver.database.windows.net")\
              .option("port","1433")\
              .option("user", "gibran")\
              .option("password","Blackggs18#3")\
              .option("database","projeto-eng-dados-databaseql")\
              .option("dbtable","TABLE_ANP_50")\
              .save()

In [ ]:
# TABELAS DIMENSÕES - STAR SCHEMA

df_regiao = spark.sql(""" SELECT DISTINCT REGIAO FROM ANP_NEW
                          ORDER BY REGIAO ASC;
""")

display(df_regiao)

In [ ]:
# Importar bibliotecas do Pyspark

from pyspark.sql.functions import *

# Criando ID incremental com "monotonically_increasing_id()"" ----> Começa no ID 0 

df_regiao_id = df_regiao.withColumn("ID_REGIAO", monotonically_increasing_id()+1) # + 1 pois começa no ID 0

display(df_regiao_id)

In [ ]:
%sql

/*

CREATE SCHEMA DW;

CREATE TABLE DW.DIM_REGIAO(
ID_REGIAO NUMERIC(1) NOT NULL,
REGIAO VARCHAR(2) NULL,
PRIMARY KEY (ID_REGIAO));
*/

In [ ]:
df_regiao_id.write.format("jdbc")\
              .mode("append")\
              .option("url","jdbc:sqlserver://projeto-eng-dados-sqlserver.database.windows.net")\
              .option("port","1433")\
              .option("user", "gibran")\
              .option("password","Blackggs18#3")\
              .option("database","projeto-eng-dados-databaseql")\
              .option("dbtable","DW.DIM_REGIAO")\
              .save()   

In [ ]:
def carga_dados(df, tabela):
    df.write.format("jdbc")\
            .mode("append")\
            .option("url","jdbc:sqlserver://projeto-eng-dados-sqlserver.database.windows.net")\
            .option("port","1433")\
            .option("user", "gibran")\
            .option("password","Blackggs18#3")\
            .option("database","projeto-eng-dados-databaseql")\
            .option("dbtable",tabela)\
            .save()  

In [ ]:
from pyspark.sql.functions import *

df_estado = spark.sql(""" SELECT DISTINCT ESTADO FROM ANP_NEW
                          ORDER BY ESTADO ASC;
""")


df_estado_id = df_estado.withColumn("ID_ESTADO", monotonically_increasing_id() +1) 


'''

CREATE TABLE DW.DIM_ESTADO(
ID_ESTADO NUMERIC(2) NOT NULL,
ESTADO VARCHAR(2) NULL,
PRIMARY KEY (ID_ESTADO));

'''

display(df_estado_id)

In [ ]:
carga_dados(df_estado_id, "DW.DIM_ESTADO")

In [ ]:
from pyspark.sql.functions import *

df_municipio = spark.sql(""" SELECT DISTINCT MUNICIPIO FROM ANP_NEW
                          ORDER BY MUNICIPIO ASC;
""")


df_municipio_id = df_municipio.withColumn("ID_MUNICIPIO", monotonically_increasing_id() +1) 


'''

CREATE TABLE DW.DIM_MUNICIPIO(
ID_MUNICIPIO NUMERIC(3) NOT NULL,
ESTADO VARCHAR(30) NULL,
PRIMARY KEY (ID_MUNICIPIO));

'''

display(df_municipio_id)

In [ ]:
carga_dados(df_municipio_id, "DW.DIM_MUNICIPIO")

In [ ]:
from pyspark.sql.functions import *

df_revenda = spark.sql(""" SELECT DISTINCT REVENDA, CNPJ 
                           FROM ANP_NEW
                           ORDER BY CNPJ ASC;
""")


df_revenda_id = df_revenda.withColumn("ID_REVENDA", monotonically_increasing_id() +1) 


'''

CREATE TABLE DW.DIM_REVENDA(
ID_REVENDA NUMERIC(5) NOT NULL,
REVENDA VARCHAR(110) NULL,
CNPJ VARCHAR(30) NULL,
PRIMARY KEY (ID_REVENDA));

'''

display(df_revenda_id)

In [ ]:
carga_dados(df_revenda_id, "DW.DIM_REVENDA")

In [ ]:
from pyspark.sql.functions import *

df_produto = spark.sql(""" SELECT DISTINCT PRODUTO
                           FROM ANP_NEW
                           WHERE PRODUTO IS NOT NULL
                           ORDER BY PRODUTO ASC;
""")


df_produto_id = df_produto.withColumn("ID_PRODUTO", monotonically_increasing_id() +1) 


'''

CREATE TABLE DW.DIM_PRODUTO(
ID_PRODUTO NUMERIC(2) NOT NULL,
PRODUTO VARCHAR(20) NULL,
PRIMARY KEY (ID_PRODUTO));

'''

display(df_produto_id)

In [ ]:
carga_dados(df_produto_id, "DW.DIM_PRODUTO")

In [ ]:
from pyspark.sql.functions import *

df_bandeira = spark.sql(""" SELECT DISTINCT BANDEIRA
                           FROM ANP_NEW
                           WHERE BANDEIRA IS NOT NULL
                           ORDER BY BANDEIRA ASC;
""")


df_bandeira_id = df_bandeira.withColumn("ID_BANDEIRA", monotonically_increasing_id() +1) 


'''

CREATE TABLE DW.DIM_BANDEIRA(
ID_BANDEIRA NUMERIC(2) NOT NULL,
BANDEIRA VARCHAR(30) NULL,
PRIMARY KEY (ID_BANDEIRA));

'''

display(df_bandeira_id)

In [ ]:
carga_dados(df_bandeira_id, "DW.DIM_BANDEIRA")

In [ ]:
from pyspark.sql.functions import *

df_unid_med= spark.sql(""" SELECT DISTINCT UNID_MED
                           FROM ANP_NEW
                           WHERE UNID_MED IS NOT NULL
                           ORDER BY UNID_MED ASC;
""")


df_unid_med_id = df_unid_med.withColumn("ID_UNID_MED", monotonically_increasing_id() +1) 


'''

CREATE TABLE DW.DIM_UNID_MED(
ID_UNID_MED NUMERIC(2) NOT NULL,
UNID_MED VARCHAR(30) NULL,
PRIMARY KEY (ID_UNID_MED));

'''

display(df_unid_med_id)

In [ ]:
carga_dados(df_unid_med_id, "DW.DIM_UNID_MED")

In [ ]:
from pyspark.sql.functions import *

df_endereco= spark.sql(""" SELECT DISTINCT NOME_RUA,
                                           NUM_RUA,
                                           COMPL_RUA,
                                           BAIRRO,
                                           CEP
                                           FROM ANP_NEW
                                           ORDER BY CEP ASC;
""")


df_endereco_id = df_endereco.withColumn("ID_ENDERECO", monotonically_increasing_id() +1) 


'''

CREATE TABLE DW.DIM_ENDERECO(
ID_ENDERECO NUMERIC(6) NOT NULL,
NOME_RUA VARCHAR(70) NULL,
NUM_RUA VARCHAR(12) NULL,
COMPL_RUA VARCHAR(128) NULL, 
BAIRRO VARCHAR(48) NULL,
CEP VARCHAR(10) NULL,
PRIMARY KEY (ID_ENDERECO));
 
'''

display(df_endereco_id)


In [ ]:
carga_dados(df_endereco_id, "DW.DIM_ENDERECO")

In [1]:
# TABELA FATO - STAR SCHEMA

df_dm_endereco = df_endereco_id.createOrReplaceTempView("DIM_ENDERECO")
df_dm_revenda = df_revenda_id.createOrReplaceTempView("DIM_REVENDA")
df_dm_estado = df_estado_id.createOrReplaceTempView("DIM_ESTADO")
df_dm_municipio = df_municipio_id.createOrReplaceTempView("DIM_MUNICIPIO")
df_dm_regiao = df_regiao_id.createOrReplaceTempView("DIM_REGIAO")
df_dm_produto = df_produto_id.createOrReplaceTempView("DIM_PRODUTO")
df_dm_bandeira = df_bandeira_id.createOrReplaceTempView("DIM_BANDEIRA")

"""

FUNÇÃO PARA CRIAR VIEWS DAS DIMENSÕES NO DATABRICKS - SIMULANDO UM AMBIENTE REAL - PEGANDO DO SQL SERVER

def leituraDB(query):
    df_leitura = spark.read\
    .format("jdbc")\
    .option("url","jdbc:sqlserver://projeto-eng-dados-sqlserver.database.windows.net")\
    .option("port","1433")\
    .option("user", "gibran")\
    .option("password","Blackggs18#3")\
    .option("database","projeto-eng-dados-databaseql")\
    .option("query", query)\
    .load()

return df_leitura   

"""

"""

CRIANDO AS VIEWS COM A FUNÇÃO leituraDB() - PEGANDO A TABELA DO SQL SERVER

Exemplo:

df_dm_endereco = leituraDB("SELECT * FROM DW.DIM_ENDERECO")

df_dm_endereco.createOrReplaceTempView("DIM_ENDERECO")

"""


In [ ]:
# Função manual 

df_med = spark.read\
    .format("jdbc")\
    .option("url","jdbc:sqlserver://projeto-eng-dados-sqlserver.database.windows.net")\
    .option("port","1433")\
    .option("user", "gibran")\
    .option("password","Blackggs18#3")\
    .option("database","projeto-eng-dados-databaseql")\
    .option("query", "SELECT * FROM DW.DIM_UNID_MED")\
    .load()



df_dm_unidade_med = df_med.createOrReplaceTempView("DIM_UNIDADE_MED")

In [ ]:
# Primeiramente faz se uma filtragem para não trazer dados nulos 

df_fato_filtro = spark.sql(""" SELECT * FROM ANP_NEW 
                               WHERE PRODUTO IS NOT NULL
                               AND REVENDA IS NOT NULL
                               AND DT_COLETA IS NOT NULL
                               AND VL_VENDA IS NOT NULL
                               AND VL_COMPRA IS NOT NULL;

""")

display(df_fato_filtro)
 

In [ ]:
df_fato_filtro.createOrReplaceTempView("FATO_FILTRO")

In [ ]:
# Tabela Fato vindo de FATO_FILTRO

df_tab_fato = spark.sql(""" SELECT A.DT_COLETA,
                            A.VL_VENDA,
                            A.VL_COMPRA,
                            B.ID_REVENDA,
                            C.ID_PRODUTO,
                            D.ID_ESTADO,
                            E.ID_MUNICIPIO,
                            F.ID_REGIAO,
                            G.ID_UNID_MED,
                            H.ID_BANDEIRA,
                            I.ID_ENDERECO
                        FROM FATO_FILTRO AS A
                        LEFT JOIN DIM_REVENDA AS B
                        ON A.REVENDA = B.REVENDA

                        LEFT JOIN DIM_PRODUTO AS C
                        ON A.PRODUTO = C.PRODUTO

                        LEFT JOIN DIM_ESTADO AS D
                        ON A.ESTADO = D.ESTADO 

                        LEFT JOIN DIM_MUNICIPIO AS E
                        ON A.MUNICIPIO = E.MUNICIPIO

                        LEFT JOIN DIM_REGIAO AS F
                        ON A.REGIAO = F.REGIAO

                        LEFT JOIN DIM_UNIDADE_MED AS G
                        ON A.UNID_MED = G.UNID_MED

                        LEFT JOIN DIM_BANDEIRA AS H
                        ON A.BANDEIRA = H.BANDEIRA 

                        LEFT JOIN DIM_ENDERECO AS I
                        ON A.CEP = I.CEP;
""")

display(df_tab_fato)

In [ ]:
# GRAVANDO A TABELA FATO NO DATA LAKE

df_tab_fato.coalesce(1)\
    .write\
    .format('parquet')\
    .mode('overwrite')\
    .save('/mnt/projetodados/refined/fato', header=True)

In [ ]:
def mover_arquivo(extensao, origem, destino, novo_nome):
    files = dbutils.fs.ls(origem)
    for i in range(0, len(files)):
        file = files[i].name

        if extensao in file:
            dbutils.fs.cp(files[i].path, destino + novo_nome + extensao)
            print('Origem é: ' + file + ' ' + 'Destino é: ' + destino + novo_nome + extensao)

In [ ]:
mover_arquivo('.parquet', '/mnt/projetodados/refined/fato/', '/mnt/projetodados/refined/fato_anp/', 'tb_fato_anp')

In [ ]:
%sql

/*
CREATE TABLE DW.FATO_ANP(
DT_COLETA DATE NOT NULL,
VL_COMPRA NUMERIC(4,3) NOT NULL,
VL_VENDA NUMERIC(4,3) NOT NULL,
ID_REVENDA NUMERIC(5) NOT NULL,
ID_PRODUTO NUMERIC(5) NOT NULL,
ID_ESTADO NUMERIC(5) NOT NULL,
ID_MUNICIPIO NUMERIC(5) NOT NULL,
ID_REGIAO NUMERIC(5) NOT NULL,
ID_UNID_MED NUMERIC(5) NOT NULL,
ID_BANDEIRA NUMERIC(5) NOT NULL,
ID_ENDERECO NUMERIC(5)NOT NULL);

*/